# Notebook Setup

In [23]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import timedelta

# pd.set_option('display.float_format', lambda x: '%.2f' % x) # suppress scientific notation
pd.set_option('display.max_rows', 150)

# Addresses and Desired Data

In [24]:
addresses = {}

addresses['FP Main'] = '0xbc49de68bcbd164574847a7ced47e7475179c76b'
addresses['FP OPEX'] = '0xc9044dd6162106edb5ece3af8a8e8b657b612e42'
addresses['HW Phase3'] = '0x45e77E6473676aB30Ed64f3815Fea7847D02D7b8'
addresses['HW Compensation'] = '0x7Af805dcd0C18Ed276f99D83197fE7a9D37B3c54'

txtypes = {}

txtypes['normal'] = 'txlist'
txtypes['erc20'] = 'tokentx'
txtypes['NFTs'] = 'tokennfttx'
txtypes['contract_interactions'] = 'txlistinternal'


# Requests

In [42]:
dfs = {}
merge = []
# get data from etherscan api
for action in txtypes:
    for address in addresses:
        payload={
            'module':'account',
            'action':txtypes[action],
            'address':addresses[address],
            'startblock':0,
            'endblock':'999999999',
            'sort':'asc',
            'apikey':'S53GR6DWMY1G9CY6X3U4Y9I9NKB8URMM5U'
        }
        r = requests.get('https://api.etherscan.io/api', params=payload)
        response = json.loads(r.text)['result']
        merge.append(pd.DataFrame.from_dict(response))

    dfs[action] = pd.concat(merge)


# Output Data

In [49]:
for df in dfs:
    dfs[df].to_csv('outputs/' + df + '.csv', index=False)
